<a href="https://colab.research.google.com/github/dksifoua/Object-Detection/blob/master/1%20-%20R-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Oct 12 19:49:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Dependencies

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')

TensorFlow version: 2.3.0


## Prepare data

In [3]:
%%time
!mkdir -p ./data
!wget --no-check-certificate \
    https://data.deepai.org/PASCALVOC2007.zip \
    -O ./data/pascal-voc-2007.zip
!unzip -q ./data/pascal-voc-2007.zip -d ./data
!rm -r ./data/pascal-voc-2007.zip

--2020-10-12 19:49:09--  https://data.deepai.org/PASCALVOC2007.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1774851628 (1.7G) [application/x-zip-compressed]
Saving to: ‘./data/pascal-voc-2007.zip’

./data/pascal-voc-2 100%[===================>]   1.65G  20.1MB/s    in 84s     

2020-10-12 19:50:34 (20.1 MB/s) - ‘./data/pascal-voc-2007.zip’ saved [1774851628/1774851628]

CPU times: user 557 ms, sys: 147 ms, total: 703 ms
Wall time: 1min 43s
